In [1]:
import argparse
import glob
import json
import os
import random
from typing import List
from concurrent.futures import ProcessPoolExecutor
from functools import partial

import numpy as np
import requests
import sentencepiece as spm
import torch
import torch.distributed as dist
from tqdm import tqdm

from tokenizer import Tokenizer

In [2]:
DATA_CACHE_DIR = "data"

In [3]:
def download_file(url: str, fname: str, chunk_size=1024):
    """发送HTTP GET请求以流式方式获取文件"""
    resp = requests.get(url, stream=True)
    
    # 获取文件的总大小（以字节为单位），默认为0如果没有提供'content-length'头信息
    total = int(resp.headers.get("content-length", 0))
    
    # 以写二进制模式打开一个文件以保存下载的内容
    with open(fname, "wb") as file, tqdm(
        desc=fname,           # 进度条前面的描述信息（通常是文件名）
        total=total,          # 总的字节数，用于设置进度条的总长度
        unit="iB",            # 进度条的单位，'iB'代表二进制字节
        unit_scale=True,      # 启用单位缩放，如KB、MB等
        unit_divisor=1024,    # 设置单位换算的除数，这里为1024
    ) as bar:
        # 逐块读取响应内容并写入文件
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)  # 写入数据块到文件
            bar.update(size)         # 更新进度条


In [4]:
def download():
    """在DATA_CACHE_DIR中创建目录，如果目录不存在则创建"""
    os.makedirs(DATA_CACHE_DIR, exist_ok=True)

    # 定义TinyStories数据集的下载URL和保存的文件名
    data_url = "https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStories_all_data.tar.gz"
    data_filename = os.path.join(DATA_CACHE_DIR, "TinyStories_all_data.tar.gz")
    
    # 检查数据集是否已经下载，如果没有下载则进行下载
    if not os.path.exists(data_filename):
        print(f"Downloading {data_url} to {data_filename}...")
        download_file(data_url, data_filename)  # 使用之前定义的download_file函数进行下载
    else:
        print(f"{data_filename} already exists, skipping download...")

    # 定义解压缩后的数据目录
    data_dir = os.path.join(DATA_CACHE_DIR, "TinyStories_all_data")
    
    # 检查数据目录是否存在，如果不存在则解压缩数据集
    if not os.path.exists(data_dir):
        os.makedirs(data_dir, exist_ok=True)  # 创建数据目录
        print(f"Unpacking {data_filename}...")
        os.system(f"tar -xzf {data_filename} -C {data_dir}")  # 使用系统命令解压缩.tar.gz文件
    else:
        print(f"{data_dir} already exists, skipping unpacking...")

    # 查找解压后的所有JSON文件，排序后获取文件名列表
    shard_filenames = sorted(glob.glob(os.path.join(data_dir, "*.json")))
    
    # 打开第一个JSON文件并读取内容
    with open(shard_filenames[0], "r") as f:
        data = json.load(f)  # 将JSON文件内容加载到变量data中
    
    print("Download done.")  # 下载完成信息
    print(f"Number of shards: {len(shard_filenames)}")  # 打印解压后数据分片的数量
    print(f"Example story:\n{data[0]}")  # 打印第一个分片中的一个示例故事

In [5]:
download()

data/TinyStories_all_data.tar.gz already exists, skipping download...
data/TinyStories_all_data already exists, skipping unpacking...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not t

In [6]:
def train_vocab(vocab_size):
    """
    Trains a custom sentencepiece tokenizer on the TinyStories dataset.
    The custom tokenizer files will be saved in DATA_CACHE_DIR/tok{N} directories,
    where N is the vocab size. This is also where the pretok .bin files will go.
    """
    # 断言词汇表大小必须为正数
    assert vocab_size > 0, "Vocab size must be positive"

    # 设置sentencepiece输出文件的前缀路径
    prefix = os.path.join(DATA_CACHE_DIR, f"tok{vocab_size}")

    # 设置用于词汇表训练的数据分片数量，数量较少以提高效率
    num_shards = 10

    # 1) 导出一大块文本作为单个文本文件tiny.txt
    tiny_file = os.path.join(DATA_CACHE_DIR, "tiny.txt")
    data_dir = os.path.join(DATA_CACHE_DIR, "TinyStories_all_data")
    shard_filenames = sorted(glob.glob(os.path.join(data_dir, "*.json")))

    print(f"Writing temporary file {tiny_file} with {num_shards} shards...")
    # 打开一个输出文件tiny.txt以写入模式
    with open(tiny_file, "w", encoding="utf-8") as of:
        # 遍历前num_shards个数据分片
        for shard in tqdm(shard_filenames[:num_shards]):
            with open(shard, "r") as f:
                data = json.load(f)  # 读取JSON文件内容
            # 从每个数据分片中提取故事文本
            for example in data:
                text = example["story"]
                text = text.strip()
                of.write(text + "\n")  # 写入每个故事到tiny.txt
    print(f"Size is: {os.path.getsize(tiny_file) / 1024 / 1024:.2f} MB")  # 打印文件大小

    # 2) 使用sentencepiece模型训练分词器
    print("Will now train the vocab...")
    spm.SentencePieceTrainer.train(
        input=tiny_file,            # 输入文本文件
        model_prefix=prefix,        # 模型输出文件的前缀
        model_type="bpe",           # 使用BPE（字节对编码）模型
        vocab_size=vocab_size,      # 设置词汇表大小
        self_test_sample_size=0,    # 自测试样本大小，设置为0表示不使用
        input_format="text",        # 输入格式为纯文本
        character_coverage=1.0,     # 字符覆盖率为1.0，表示全部字符都覆盖
        num_threads=os.cpu_count(), # 使用的线程数为CPU的核心数
        split_digits=True,          # 拆分数字
        allow_whitespace_only_pieces=True, # 允许只有空格的分片
        byte_fallback=True,         # 启用字节回退
        unk_surface=r" \342\201\207 ",  # 未知字符的表示方式
        normalization_rule_name="identity" # 使用identity正则化规则
    )

    # 3) 可选的清理操作，询问用户是否删除临时文件
    dec = input(f"Delete the temporary file {tiny_file}? [y/N] ")
    if dec.lower() == "y":
        os.remove(tiny_file)  # 删除临时文件
        print(f"Deleted {tiny_file}")

    # 打印训练完成的信息
    print(f"Trained tokenizer is in {prefix}.model")
    print("Done.")


In [7]:
train_vocab(2048)

Writing temporary file data/tiny.txt with 10 shards...


100%|██████████| 10/10 [00:36<00:00,  3.65s/it]
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/tiny.txt
  input_format: text
  model_prefix: data/tok2048
  model_type: BPE
  vocab_size: 2048
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surfa

Size is: 739.57 MB
Will now train the vocab...


: 0
  differential_privacy_noise_level: 0
  differential_privacy_clipping_threshold: 0
}
normalizer_spec {
  name: identity
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  escape_whitespaces: 1
  normalization_rule_tsv: 
}
denormalizer_spec {}
trainer_interface.cc(353) LOG(INFO) SentenceIterator is not specified. Using MultiFileSentenceIterator.
trainer_interface.cc(185) LOG(INFO) Loading corpus: data/tiny.txt
trainer_interface.cc(147) LOG(INFO) Loaded 1000000 lines
trainer_interface.cc(147) LOG(INFO) Loaded 2000000 lines
trainer_interface.cc(147) LOG(INFO) Loaded 3000000 lines
trainer_interface.cc(124) LOG(WARNING) Too many sentences are loaded! (3992613), which may slow down training.
trainer_interface.cc(126) LOG(WARNING) Consider using --input_sentence_size=<size> and --shuffle_input_sentence=true.
trainer_interface.cc(129) LOG(WARNING) They allow to randomly sample <size> sentences from the entire corpus.
trainer_interface.cc(409) LOG(INFO) Loaded all 3992613 sentences
train

Trained tokenizer is in data/tok2048.model
Done.


In [ ]:
def get_tokenizer_model_path(vocab_size):
    """
    Returns path to the sentencepiece tokenizer model for a given vocab size
    vocab_size = 0 designates the default Llama 2 tokenizer, in that case
    None is returned.
    """
    if vocab_size == 0:
        return None
    else:
        return os.path.join(DATA_CACHE_DIR, f"tok{vocab_size}.model")

class Task:

    @staticmethod
    def iter_batches(batch_size, device, num_workers=0, **dataset_kwargs):
        ds = PretokDataset(**dataset_kwargs)
        dl = torch.utils.data.DataLoader(
            ds, batch_size=batch_size, pin_memory=True, num_workers=num_workers
        )
        for x, y in dl:
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            yield x, y

In [ ]:
def process_shard(args, vocab_size):
    shard_id, shard = args  # 解包输入参数，shard_id是分片ID，shard是分片文件路径
    tokenizer_model = get_tokenizer_model_path(vocab_size)  # 获取分词器模型的路径
    enc = Tokenizer(tokenizer_model)  # 初始化分词器对象
    with open(shard, "r") as f:
        data = json.load(f)  # 读取JSON文件内容
    
    all_tokens = []  # 用于存储所有的分词结果
    for example in tqdm(data, position=shard_id):  # 遍历分片中的每个故事
        text = example["story"]
        text = text.strip()  # 去除前后空格
        tokens = enc.encode(text, bos=True, eos=False)  # 对文本进行编码，添加BOS（句子开始）标记
        all_tokens.extend(tokens)  # 将编码后的令牌添加到all_tokens列表中
    
    # 将所有令牌转换为uint16类型的NumPy数组
    all_tokens = np.array(all_tokens, dtype=np.uint16)
    
    # 计算输出文件名
    if vocab_size == 0:
        # 如果使用的是Llama 2模型，将分词后的文件保存在同一目录
        tokenized_filename = shard.replace(".json", ".bin")
    else:
        # 将二进制文件保存到新的tok{N}目录中
        bin_dir = os.path.join(DATA_CACHE_DIR, f"tok{vocab_size}")
        shard_basename = os.path.basename(shard)
        bin_basename = shard_basename.replace(".json", ".bin")
        tokenized_filename = os.path.join(bin_dir, bin_basename)
    
    # 将令牌写入二进制文件
    with open(tokenized_filename, "wb") as f:
        f.write(all_tokens.tobytes())
    
    # 计算平均序列长度（使用BOS=1分隔序列）
    avg_seq_len = all_tokens.size / ((all_tokens == 1).sum())
    print(f"Saved {tokenized_filename}, average seqlen: {avg_seq_len:.2f}")
